In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [14]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value', 'key', 'attention.output.dense'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 741,124 || all params: 125,389,832 || trainable%: 0.5911


In [15]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [16]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.363300,1.315394,0.776563
200,0.945200,0.387021,0.871875
300,0.384400,0.356146,0.885938
400,0.295200,0.344582,0.896875
500,0.316700,0.333270,0.896875
600,0.303900,0.329503,0.901563
700,0.310200,0.332190,0.903125
800,0.328000,0.316057,0.906250
900,0.304500,0.313749,0.901563
1000,0.294000,0.310571,0.904687


In [18]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-1200'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [01:00<00:00,  4.13it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-1200.csv


In [17]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3633,1.315394,0.378750,0.776563,0.000048,0.013405,0.047906,1.036420,-0.397813,0.487726
1,200,0.9452,0.387021,0.771429,0.871875,0.000045,0.026810,0.558179,2.442245,-0.100446,0.884793
2,300,0.3844,0.356146,0.873661,0.885938,0.000043,0.040214,0.028254,1.079332,-0.012277,0.986143
3,400,0.2952,0.344582,0.896875,0.896875,0.000040,0.053619,-0.049382,0.856690,0.000000,1.000000
4,500,0.3167,0.333270,0.894196,0.896875,0.000037,0.067024,-0.016570,0.950280,-0.002679,0.997013
5,600,0.3039,0.329503,0.897768,0.901563,0.000035,0.080429,-0.025603,0.922298,-0.003795,0.995791
6,700,0.3102,0.332190,0.902232,0.903125,0.000033,0.093834,-0.021990,0.933804,-0.000893,0.999011
7,800,0.3280,0.316057,0.891518,0.906250,0.000030,0.107239,0.011943,1.037788,-0.014732,0.983744
8,900,0.3045,0.313749,0.901339,0.901563,0.000028,0.120643,-0.009249,0.970521,-0.000223,0.999752
9,1000,0.2940,0.310571,0.900446,0.904687,0.000025,0.134048,-0.016571,0.946643,-0.004241,0.995312
